# Проект по SQL

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.

Ваша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Ваша первая задача как аналитика — проанализировать базу данных.
В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

## Описание данных

Таблица `books`  
Содержит данные о книгах:
- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

Таблица `authors`  
Содержит данные об авторах:  
- `author_id` — идентификатор автора;
- `author` — имя автора.

Таблица `publishers`  
Содержит данные об издательствах:  
- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

Таблица `ratings`  
Содержит данные о пользовательских оценках книг:  
- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

Таблица `reviews`  
Содержит данные о пользовательских обзорах:  
- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя автора обзора;
- `text` — текст обзора.

## Задачи
- Посчитайте, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитайте количество обзоров и среднюю оценку;
- Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
- Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
- Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

## Обзор данных

In [ ]:
# Импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

In [ ]:
# Установим параметры
db_config = {
    'user': 'praktikum_student', # имя пользователя
    'pwd': 'Sdf4$2;d-d30pp', # пароль
    'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
    'port': 6432, # порт подключения
    'db': 'data-analyst-final-project-db' # название базы данных
} 

In [ ]:
# Пропишем строку подключения
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(
    db_config['user'],
    db_config['pwd'],
    db_config['host'],
    db_config['port'],
    db_config['db']
)

In [ ]:
# Сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode': 'require'})

In [ ]:
def output(query):
    """ Функция выводит данные по запросу
    """
    
    return pd.io.sql.read_sql(query, con=engine)

In [ ]:
# Выведем первые пять строк каждой таблицы
for item in 'books', 'authors', 'publishers', 'ratings', 'reviews':
    query = 'SELECT * FROM ' + item + ' LIMIT 5;'
    display(pd.io.sql.read_sql(query, con=engine))
    print('********************************************************************************************************') 

### Посмотрим типы данных, пропуски, дубликаты

In [ ]:
# Присвоим таблицам имена
books = ''' SELECT * FROM books'''
authors = ''' SELECT * FROM authors'''
publishers = ''' SELECT * FROM publishers'''
ratings = ''' SELECT * FROM ratings'''
reviews = ''' SELECT * FROM reviews'''

In [ ]:
# Выведем информацию о таблицах
for df in output(books), output(authors), output(publishers), output(ratings), output(reviews):
    display(df.info(), df.duplicated().sum(), df.isna().sum())
    print('****************************************')

Вывод:
- в таблицах не обнаружены пропуски и дубликаты;
- в таблице `books`, в столбце `publication_date` изменим тип данных на datetime.

In [ ]:
books = pd.io.sql.read_sql(books, con=engine)
books['publication_date'] = pd.to_datetime(books['publication_date'])

In [ ]:
# Проверим изменения
books.info()

ok

In [ ]:
# Выведем SQL-запрос, отсортируем записи в порядке убывания 
# по полю `publication_date`
publication_date = '''
    SELECT title,
    publication_date
    FROM books
    ORDER BY publication_date DESC 
'''

output(publication_date)

## Исследовательский анализ данных

- Посчитаем количество книг вышедших после 1 января 2000 года

In [ ]:
number_of_books = '''
    SELECT COUNT(book_id) AS books_count
    FROM books
    WHERE publication_date::date > '2000-01-01'
'''

output(number_of_books)

Вывод:
- после 1 января 2000 года было опубликовано 819 книг.

---



- Для каждой книги посчитаем количество обзоров и среднюю оценку

In [ ]:
# Объединим таблицы через LEFT JOIN
average_rating = '''
    SELECT t.title,
    COUNT(t.title),
    ROUND(AVG, 1) AS avg
    FROM(
        SELECT b.book_id, b.title,
        AVG(rat.rating)
        FROM books b
        LEFT JOIN ratings AS rat ON b.book_id = rat.book_id
        GROUP BY b.book_id
    ) AS t
    LEFT JOIN reviews AS rev ON t.book_id = rev.book_id
    GROUP BY t.book_id, t.title, avg
    ORDER BY COUNT(t.title) DESC
'''

output(average_rating)

Вывод:
- наибольшее количество (7 шт.) обзоров у книги `Twilight (Twilight #1)` и её средняя оценка 3.66.

---

- Определим издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключим из анализа брошюры

In [ ]:
# Соединим таблицы методом JOIN
most_books = '''
SELECT publisher AS p,
COUNT(publishers) AS p_number
FROM publishers
JOIN books ON publishers.publisher_id = books.publisher_id
WHERE num_pages > 50
GROUP BY p
ORDER BY p_number DESC
LIMIT 10;
'''

output(most_books)

Вывод:
- издательство Penguin Books выпустило 42 книги толще 50 листов.

---

- Определим автора с самой высокой средней оценкой книг — учитыввем только книги с 50 и более оценками

In [ ]:
# Используем две таблицы `books` и `ratings` через объединение INNER JOIN 
# с использованием подзапроса WHERE IN
author_highest_avg_rating = '''
    SELECT a.author,
    ROUND(AVG(rat.rating),2),
    COUNT(b.title)
    FROM books AS b
    INNER JOIN authors AS a ON b.author_id = a.author_id
    INNER JOIN ratings AS rat ON b.book_id = rat.book_id
    WHERE b.book_id IN
        (SELECT b.book_id
        FROM books AS b
        INNER JOIN ratings AS rat on b.book_id = rat.book_id
        GROUP BY b.book_id
        HAVING COUNT(rat.rating) >= 50
        )
    GROUP BY a.author
    ORDER BY AVG(rat.rating) DESC; 
''' 

output(author_highest_avg_rating)

Вывод:
- автором с самой высокой средней оценкой книг является `J.K. Rowling/Mary GrandPré`.

---

- Посчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [ ]:
user_reviews = '''
    SELECT ROUND(AVG(count), 2) AS average
    FROM(
        SELECT COUNT(text)
        FROM reviews
        FULL OUTER JOIN(
            SELECT username,
            COUNT(rating_id) AS cnt
            FROM ratings
            GROUP BY username) AS cnt_rating ON cnt_rating.username = reviews.username
            WHERE cnt > 50
            GROUP BY reviews.username
            ) AS count
'''

output(user_reviews)

Вывод:
- среднее количество обзоров от авторов поставивших более 50 оценок - 24.33

---

## Итоговый вывод:
- после 1 января 2000 года было опубликовано 819 книг;
- наибольшее количество (7 шт.) обзоров у книги `Twilight (Twilight #1)` и её средняя оценка 3.66;
- издательство Penguin Books выпустило 42 книги толще 50 листов;
- автором с самой высокой средней оценкой книг является `J.K. Rowling/Mary GrandPré`;
- среднее количество обзоров от авторов поставивших более 50 оценок - 24.33.